In [1]:
%cd /home/enzo/Documents/pickomino/rust_library

/home/enzo/Documents/pickomino/rust_library


In [2]:
%pip install maturin

  Using cached maturin-1.3.1-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.musllinux_1_1_x86_64.whl (9.5 MB)
  Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
!maturin build --release

🔗 Found pyo3 bindings
🐍 Found CPython 3.10 at /home/enzo/miniconda3/deepghost_env/bin/python3
   Compiling pyo3-build-config v0.20.0
   Compiling pyo3-ffi v0.20.0=====>      ] 35/43: pyo3-build-config           
   Compiling pyo3 v0.20.0
   Compiling rust_library v0.1.0 (/home/enzo/Documents/pickomino/rust_library)
  --> src/lib.rs:12:17
   |
12 | use std::time::{Duration, Instant};
   |                 ^^^^^^^^  ^^^^^^^
   |
   = note: `#[warn(unused_imports)]` on by default

   --> src/lib.rs:149:17
    |
149 |             let mut result: usize = n*factorial(n-1, factorial_cache);
    |                 ----^^^^^^
    |                 |
    |                 help: remove this `mut`
    |
    = note: `#[warn(unused_mut)]` on by default

   --> src/lib.rs:178:13
    |
178 |         let sum: f64 = vec_coefficients.iter().sum();
    |             ^^^ help: if this is intentional, prefix it with an underscore: `_sum`
    |
    = note: `#[warn(unused_variables)]` on by default

   --> src/

In [5]:
%pip uninstall -y rust_library --break-system-packages

Found existing installation: rust_library 0.1.0
Uninstalling rust_library-0.1.0:
  Successfully uninstalled rust_library-0.1.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install /home/enzo/Documents/pickomino/rust_library/target/wheels/rust_library-0.1.0-cp311-cp311-manylinux_2_34_x86_64.whl --break-system-packages

Defaulting to user installation because normal site-packages is not writeable
Processing /home/enzo/Documents/pickomino/rust_library/target/wheels/rust_library-0.1.0-cp311-cp311-manylinux_2_34_x86_64.whl
Note: you may need to restart the kernel to use updated packages.


In [2]:
import rust_library

In [4]:
rewards = [0. for _ in range(21)] + [1.]*4 + [2.]*4 + [3.]*4 + [4.]*4 + [0.]*4

In [5]:
rewards[37]

0.0

In [6]:
current_score = 0
picked_dice = [0,0,0,0,0,0]
remaining_dice = 8
throw_result = 0
rewards = [0. for _ in range(21)] + [1.]*4 + [2.]*4 + [3.]*4 + [4.]*4 + [0.]*4
leaving_penalty = 0.

result = rust_library.compute_best_path_py(
    current_score,
    picked_dice,
    remaining_dice,
    throw_result,
    rewards,
    leaving_penalty
)

In [12]:
current_score = 9
picked_dice = [0,0,1,0,0,0]
remaining_dice = 5
throw = (3,1,0,1,2,0,1)
throw_result = dict_index[throw]
rewards = [0. for _ in range(21)] + [1.]*4 + [2.]*4 + [3.]*4 + [4.]*4 + [0.]*4
leaving_penalty = 0.

result = rust_library.compute_best_path_py(
    current_score,
    picked_dice,
    remaining_dice,
    throw_result,
    rewards,
    leaving_penalty
)
value = rust_library.position_value_py(current_score, picked_dice, remaining_dice, rewards, leaving_penalty)
print(result)
print(value)

(1.0007739864349956, 5, True, None)
1.1112980203210665


In [13]:
%timeit value = rust_library.position_value_py(current_score, picked_dice, remaining_dice, rewards, leaving_penalty)

244 µs ± 23 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
result

(4.0, 5, True, None)

In [8]:
value = rust_library.position_value_py(current_score, picked_dice, remaining_dice, rewards, leaving_penalty)
print(value)
print(result)

1.6447296740400994
(4.0, 5, True, None)


In [5]:
value

1.752055974787671

In [12]:
result

(2.0258543893747984, 100, False, None)

In [11]:
%timeit result = rust_library.compute_best_path_py( current_score, picked_dice, remaining_dice, throw_result, rewards, leaving_penalty)

335 µs ± 565 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
import torch
import torch.nn as nn
import numpy as np
import gymnasium as gym
from gymnasium.spaces import Box, Dict, Discrete, MultiBinary, MultiDiscrete
import pygame
from gymnasium import spaces

On doit d'abord créer l'espace d'états.

In [9]:
class Policy_Net(nn.Module):
    def __init__(self, input_dimension, hidden_dimensions, out_dimension=12):
        super.__init__()
        self.embedding = torch.nn.Linear(input_dimension, hidden_dimensions[0])
        module_list = [self.embedding
        ]
        for i in range(len(hidden_dimensions) - 1):
            module_list.append(nn.ReLU())
            module_list.append(nn.Linear(hidden_dimensions[i], hidden_dimensions[i+1]))
        # 6 choices of dice + whether to continue = 12 possibilities among which to choose
        module_list.append(nn.ReLU())
        module_list.append(nn.Linear(hidden_dimensions[-1], out_dimension))
        module_list.append(nn.Softmax(out_dimension))
        self.sequence = nn.Sequential(*module_list)
    
    def forward(self, x):
        return self.sequence(x)


In [9]:
from scipy.special import binom
def bn(u,v):
    return int(binom(u,v))
list_points = [bn(i,5) for i in range(13,4, -1)]
list_intervals = [0]
for i in range(len(list_points)):
    list_intervals.append(list_intervals[-1] + list_points[i])
def index_partition():
    """Via la hockey stick identity, on construit l'ensemble des jetés de dés."""
    dict_index = {}
    for i_0 in range(9):
        for i_1 in range(9-i_0):
            for i_2 in range(9-i_0-i_1):
                for i_3 in range(9-i_0-i_1-i_2):
                    for i_4 in range(9-i_0-i_1-i_2-i_3):
                        for i_5 in range(9-i_0-i_1-i_2-i_3-i_4):
                            i_6 = 8-i_0-i_1-i_2-i_3-i_4-i_5
                            dict_index[((i_0,i_1,i_2,i_3,i_4,i_5,i_6))] = len(dict_index)
    return dict_index

dict_index = index_partition()
dict_number = {v: k for k, v in dict_index.items()}
list_numbers = list(dict_number.values())
list_throws = [x[1:] for x in list_numbers]

In [2]:
list_intervals

[0, 1287, 2079, 2541, 2793, 2919, 2975, 2996, 3002, 3003]

In [7]:
list_numbers[2793]

(4, 0, 0, 0, 0, 0, 4)

In [5]:
dict_index[(0,2,1,0,3,1,1)]

925

In [16]:
d = Discrete(41)

In [34]:
np.array([0]*20+[1]*4+[2]*4+[3]*4+[4]*4).shape

(36,)

In [ ]:
class PickominoEnv(gym.Env):
    def __init__(self, value_size=41,
                 dice_picked_size=6,
                 dice_remaining_size=9,
                 throw_size=3003,
                 pickomino_states_dim=16,
                 rewards=np.array([0.]*21+[1.]*4+[2.]*4+[3.]*4+[4.]*4 + [0.]*4)):
        

        """The player is trying to maximize player_total,
        which will determine whether he wins at the end of the game."""
        self.state_space = Dict({"value": Discrete(41),
                    "dice_picked": MultiBinary(6),
                    "dice_remaining": Discrete(9),
                    "throw": Discrete(3003),
                    "pickomino_states": Box(-1,2,16, dtype=np.int), # 2: hidden, 1: belongs to adversary, 0: not picked, -1: belongs to me, 
                    "rewards": Box(0,4,41),
                    "adversary_total": Discrete(41),
                    "player_total": Discrete(41),
            })
        self.current_state = {"value": np.int32(0),
                              "dice_picked": np.array[0,0,0,0,0,0],
                              "dice_remaining": np.int32(8),
                              "throw": 0,
                              "pickomino_states": np.array([0]*16),
                              "rewards": np.array([0]*20+[1]*4+[2]*4+[3]*4+[4]*4 + [0]*4),
                              "adversary_total": np.int32(0),
                              "player_total": np.int32(0),}
        self.actions = Dict({"choose_dice": Discrete(6),
                             "stop_playing": MultiBinary(1),
        }
        )
        self.big_penalty = -1000 # Penality in case we choose a choice that makes no sense
    
    def _create_reward_adversary(self,alpha,beta):
        index_owned_by_adversary = -1
        index_owned_by_me = -2
        for index, value in enumerate(self.current_state["pickomino_states"]):
            if value == 1:
                index_owned_by_adversary = index
            elif value == -1:
                index_owned_by_me = index
        c = alpha*self.current_state["rewards"][index_owned_by_adversary]#alpha*number of worms on the tile owned by adversary
        rewards_adversary = []
        for i in range(41):
            if i <= 20:
                rewards_adversary.append(-c)
            if self.current_state["pickomino_states"][i] == 2:
                rewards_adversary.append(max(rewards_adversary))
            elif self.current_state["pickomino_states"][i] == 1:
                rewards_adversary.append(max(rewards_adversary))
            elif self.current_state["pickomino_states"][i] == 0:
                rewards_adversary.append(max(max(rewards_adversary), self.current_state["rewards"][i]))
            elif self.current_state["pickomino_states"][i] == -1:
                rewards_adversary.append(-2*beta*self.current_state["rewards"][i])
        return rewards_adversary
    def _simulate_adversary(self, alpha, beta):
        """The adversary implements the best strategy as implemented in Rust.
        We need to give it the appropriate reward, give random draws, and compute
        its choice for each draw. The final state is the state in which it has 0 dice.
        
        
        Several things have to be updated at the end: 
        - the states of the pickominos
        - the total of the adversary
        - the total of the player
        """
        current_score = 0
        picked_dice = [0,0,0,0,0,0]
        remaining_dice = 8
        interval_start = 8-remaining_dice
        throw_result = np.random.randint(list_intervals[interval_start], list_intervals[interval_start+1])

        leaving_penalty = 0.
        rewards_adversary = self._create_reward_adversary(alpha, beta)
        continue_playing = True

        while continue_playing:
            result = rust_library.compute_best_path_py(
                current_score,
                picked_dice,
                remaining_dice,
                throw_result,
                rewards,
                leaving_penalty
            )
            (max_expected_reward, dice_choice, throw_again, pickomino_choice) = result
    def _get_obs(self):
        dice_remaining = self.current_state["dice_remaining"]
        low = list_intervals[8-dice_remaining]
        high = list_intervals[8-dice_remaining+1]
        throw = np.random.randint(low, high)
        self.current_state["throw"] = throw
        return None

    def _update_space(self, new_value, new_dice_picked, new_dice_remaining, new_throw, new_pickomino_states):
        self.current_state = {"value": new_value,
                              "dice_picked": new_dice_picked,
                              "dice_remaining": new_dice_remaining,
                              "throw": new_throw,
                              "pickomino_states": new_pickomino_states}
    def reset(self, seed=111, options=None):
        # We need the following line to seed self.np_random
        super().reset(seed=seed)

        # Choose the agent's location uniformly at random
        self._agent_location = self.np_random.integers(0, self.size, size=2, dtype=int)

        # We will sample the target's location randomly until it does not coincide with the agent's location
        self._target_location = self._agent_location
        while np.array_equal(self._target_location, self._agent_location):
            self._target_location = self.np_random.integers(
                0, self.size, size=2, dtype=int
            )

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, info
    def step(self, action):
        # We have to compute what happens when we take a certain action in a certain state
        # 1. We throw the dice at the beginning.
        self._get_obs() # Updates the throw
        # 2. We now have to pick a dice and choose whether or not to throw again
        terminated = False
        reward = 0
        state_tensor = 
        throw_again, number_chosen = self.policy(state_tensor)
        throw_values = list_throws[self.current_state["throw"]]
        if self.current_state["throw_values"][number_chosen] == 0 or self.current_state["dice_picked"][number_chosen]!=0:
            reward = self.big_penalty
            terminated = True
            info = f"Terminated with {reward=}, {self.current_state}"
        else:
            number_of_chosen_dice = self.current_state["throw"][number_chosen]
            new_value = max(number_chosen + 1, 5)*number_of_chosen_dice
            new_dice_picked = self.current_state["dice_picked"]
            new_dice_picked[number_chosen] = 1
            new_dice_remaining = self.current_state["dice_remaining"] - number_of_chosen_dice
            new_throw = self.current_state["throw"]
            new_pickomino_states = self.current_state["pickomino_states"]
            self._update_space(new_value, new_dice_picked)
            info = f"current_state: {self.current_state}"
            if new_dice_remaining == 0 or throw_again == 0:
                # We attribute the reward and terminate the game
                if self.current_state["dice_picked"][5]! = 0 #On a au moins un pickomino
                    reward = 
        observation = self.current_state
        truncated = False
        return observation, reward, terminated, truncated, info

On peut essayer un pre-training à l'aide des trajectoires optimales calculées pour la partie I